# Baseball  Regression Modeling Model Selection

## Objectivos de Aprendizaje

Correr regresión lineal básica utilizando regularización L1 y aprender algo de selección de variables
    * ejecutar una regresión lineal simple de diferencias cuadradas
    * analisis del output
    * correr regresión L1 e identificar mejor parámetro de regularización
    * interpretar parámetros del modelo

## Imports

In [ ]:
import pandas as pd
import statsmodels.api as sms
import sklearn.linear_model as lm

In [ ]:
%matplotlib inline

## Obtener Data y generar un Subset de Data

In [ ]:
# leer archivo csv y guardar dataframe
df = pd.read_csv('../data/baseball_data.csv')

# generar subset removiendo filas con valores NULL
bix = df.notnull().all(axis=1)
df = df[bix]

## Primer Modelo de Regresión utilizando todas las variables

El objetivo de esta primera etapa es poder analizar como se relacionan todas las variables con el target (variable/atributo objetivo). Desafortunadamente, esto falla debido a la alta co-relación entre algunas variables.

In [ ]:
model = sms.OLS(df.salary_in_thousands_of_dollars, sms.add_constant(df.iloc[:, 1:]))

In [ ]:
result = model.fit()

In [ ]:
print(result.summary())

#### Notas:
    * Con este enfoque, logramos obtener un R^2 bastante alto desde el inicio 0.71, esto no siempre es bueno.
    * Mediante este resumen estadístico, se puede comenzar a observar cuales variables son 
    potencialmente significativas
        - on_base_percentage
        - number_of_runs
        - number_of_runs_batted_in
        - number_of_strike_outs
        - number_of_stolen_bases
        - indicator_of_free_agency_eligibility
        - indicator_of_free_agent_in_1991_1992
        - indicator_of_arbitration_eligibility
        - indicator_of_arbitration_in_1991_1992
    * Aun hay bastantes variables, como podemos discernir cuales usar y no usar en el modelo?
    * Warning [2] indica que probablemente existe una alta colinelidad entre algunas variables.

## Selección del Modelo utilizando Regresión L1

Notas:
        * entrenar modelo con valores de alpha entre 2^-15 y 2^15 (parámetro de regularización) 
        * esto forzará un número significativo de parámetros a cero
        * las variables asociadas a parámetros distintos de cero, serán las variables seleccionadas
        * escogeremos el modelo con menores aic y bic 
        (Akaike Information Criterion | Bayesian Information Criterion)
        * finalmente re-entrenarémos el "mejor modelo" e interpretarémos las variables

In [ ]:
model = sms.OLS(df.salary_in_thousands_of_dollars, sms.add_constant(df.iloc[:, 1:]))

In [ ]:
# declare lists for storage
nums = list()
parmslist = list()

# loop from k = -15 to k = 15 increasing the value of the 
# penalization parameter calculated 2**k
for k in range(-15, 15):
    alpha = 2**k
    
    # fit regression
    reg_results = model.fit_regularized(alpha=alpha)
    
    # get aic and bic
    aic = reg_results.aic
    bic = reg_results.bic
    
    # count the number non zero parameters
    parmct = (reg_results.params != 0).sum()
    
    # get rsqrd
    rsqr = reg_results.rsquared
    
    # collect the list of nonzero parameters
    parms = reg_results.params[(reg_results.params != 0)].index.tolist()
    
    # store values
    parmslist.append(parms)
    nums.append((alpha, aic, bic, rsqr, parmct))

In [ ]:
modelsdf = pd.DataFrame(nums, columns=['alpha', 'aic', 'bic', 'rsqr', 'parmsct'])

In [ ]:
modelsdf[['aic', 'bic']].plot()

#### Nota: 
Valores de alpha = 0 mantienen aic y bic en sus mínimos, por lo que no es necesario setear parámetro de regularización, i.e., alpha = 0

In [ ]:
i = 1
for lst in parmslist:
    strn = ''
    for item in lst:
        strn += item + ', '
    print ("modelo " + str(i) + ": " + strn)
    i += 1

## Re-run Regresión en "Mejor" Modelo

##### Modelo 1
Nota: Fijarse en modelo 23 y 22

In [ ]:
model = sms.OLS(df.salary_in_thousands_of_dollars, 
                sms.add_constant(
                    df[[
                        'number_of_runs', 
                        'number_of_home_runs', 
                        'number_of_runs_batted_in',
                        'number_of_walks',
                        'number_of_stolen_bases',
                        'indicator_of_free_agency_eligibility',
                        'indicator_of_arbitration_eligibility' 
                        ]]))

In [ ]:
results = model.fit()

In [ ]:
print(results.summary())

Notas:
    * Usando este "mejor modelo", re-entrenamos modelo sin regularización L1 (i.e., alpha = 0)
    * En este primer re-entrenamiento, las variables number_of_runs, number_of_home_runs y number_of_walks no son significativas, por lo que para el próximo re-entrenamiento podemos omitir estas variables.

##### Modelo 2

In [ ]:
model = sms.OLS(df.salary_in_thousands_of_dollars, 
                sms.add_constant(
                    df[[
                        'number_of_runs_batted_in',
                        'number_of_stolen_bases',
                        'indicator_of_free_agency_eligibility',
                        'indicator_of_arbitration_eligibility' 
                        ]]))

In [ ]:
results = model.fit()

In [ ]:
print(results.summary())

Interpretación:
    * para variables continuas, una forma de interpretar los coeficientes lineales son
        - una unidad de cambio en number_of_runs_batted_in corresponde a 18,700 dolares de incremento en el salario del atleta.
        - una unidad de cambio en number_of_stolen_bases corresponde a 11,400 dolares de incremento en el salario.
        
    * para variables binarias, podemos interpretar los coefficientes de manera similar
        - cuando un atleta es libre de ataduras contractuales, vemos un incremento promedio del salario en 1,330,000 dolares.
        - cuando un atleta tiene capacidad de solicitar arbitraje salarial, se observa un incremento promedio de 865,000 dolares en su salario.